In [ ]:
%load_ext autoreload
%autoreload 2

### preprocessing
- drop NaN values
- removed 3rd gender (n=1)
- merge rare classes
- drop classes with less than 25 samples

In [ ]:
# relative imports
from uncertainty_aware_diagnosis import ICD10data

# absolute imports

# paths
train_csv = "./data/lbz-train.csv"
val_csv = "./data/lbz-val.csv"
test_csv = "./data/lbz-test.csv"

In [ ]:
target = "icd10_main_code"
categorical = [
    "zorginstellingnaam",
    "gender",
    "clinical_specialty",
    "DBC_specialty_code",
    "DBC_diagnosis_code",
    "icd10_subtraject_code",
]
numerical = ["age"]

train = ICD10data(
    csv_path=train_csv,
    numerical=numerical,
    categorical=categorical,
    high_card=[],
    target=target,
    dropna=True,
    use_embedding=False,
)

print(f"Number of icd10 classes: {len(train.classes)}")